In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import time

from datetime import datetime, timezone
from tqdm.notebook import tqdm
from config import column_names,schedule
from influxdb_client import InfluxDBClient
from concurrent.futures import ThreadPoolExecutor, as_completed


/Users/orentapiero/projects/Crypto/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# InfluxDB configuration
bucket = "crypto"
org = os.environ["INFLUXDB_ORG"]
token = os.environ["INFLUXDB_TOKEN"]
url = os.environ["INFLUXDB_URL"]

In [3]:
def get_ohlcv_aggregated(symbol="BTCUSDT", start_date=None, end_date=None, interval="1h"):
    """
    Fetch OHLCV data with proper OHLCV aggregation from InfluxDB within a date range
    
    Args:
        symbol (str): Trading pair symbol (default: "BTCUSDT")
        start_date (str or datetime): Start date (default: 7 days ago)
        end_date (str or datetime): End date (default: now)
        interval (str): Time interval for aggregation (default: "1h")
    
    Returns:
        pandas.DataFrame: DataFrame with timestamp index and columns [open, high, low, close, volume]
    """
    
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = client.query_api()
    
    # Handle date formatting
    def format_date(date_input):
        if date_input is None:
            return None
        
        if isinstance(date_input, str):
            if len(date_input) == 10 and date_input.count('-') == 2:
                date_input += "T00:00:00Z"
            elif not date_input.endswith('Z'):
                date_input += "Z"
            return date_input
        elif isinstance(date_input, datetime):
            return date_input.strftime('%Y-%m-%dT%H:%M:%SZ')
        else:
            raise ValueError(f"Unsupported date format: {date_input}")
    
    # Set default dates if not provided
    if start_date is None:
        start_str = "-7d"
    else:
        start_str = format_date(start_date)
    
    if end_date is None:
        end_str = "now()"
    else:
        end_str = format_date(end_date)
    
    # Build the time range part of the query
    if start_date is None:
        range_clause = f'|> range(start: {start_str})'
    else:
        if end_date is None:
            range_clause = f'|> range(start: {start_str})'
        else:
            range_clause = f'|> range(start: {start_str}, stop: {end_str})'
    
    query = f'''
    // Get all base data first
    base_data = from(bucket: "{bucket}")
    {range_clause}
    |> filter(fn: (r) => r._measurement == "klines")
    |> filter(fn: (r) => r.symbol == "{symbol}")
    |> filter(fn: (r) => r._field == "open" or r._field == "high" or r._field == "low" or r._field == "close" or r._field == "volume")
    
    // Aggregate OPEN: first value in each time window
    open_data = base_data
    |> filter(fn: (r) => r._field == "open")
    |> aggregateWindow(every: {interval}, fn: first, createEmpty: false)
    |> set(key: "_field", value: "open")
    
    // Aggregate HIGH: maximum value in each time window
    high_data = base_data
    |> filter(fn: (r) => r._field == "high")
    |> aggregateWindow(every: {interval}, fn: max, createEmpty: false)
    |> set(key: "_field", value: "high")
    
    // Aggregate LOW: minimum value in each time window  
    low_data = base_data
    |> filter(fn: (r) => r._field == "low")
    |> aggregateWindow(every: {interval}, fn: min, createEmpty: false)
    |> set(key: "_field", value: "low")
    
    // Aggregate CLOSE: last value in each time window
    close_data = base_data
    |> filter(fn: (r) => r._field == "close")
    |> aggregateWindow(every: {interval}, fn: last, createEmpty: false)
    |> set(key: "_field", value: "close")
    
    // Aggregate VOLUME: sum all volumes in each time window
    volume_data = base_data
    |> filter(fn: (r) => r._field == "volume")
    |> aggregateWindow(every: {interval}, fn: sum, createEmpty: false)
    |> set(key: "_field", value: "volume")
    
    // Union all aggregated data and pivot
    union(tables: [open_data, high_data, low_data, close_data, volume_data])
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> sort(columns: ["_time"])
    |> keep(columns: ["_time", "open", "high", "low", "close", "volume"])
    '''
    
    try:
        result = query_api.query_data_frame(query)
        
        if not result.empty:
            # Convert time to datetime
            result['_time'] = pd.to_datetime(result['_time'])
            
            # Ensure numeric columns are properly typed
            numeric_columns = ['open', 'high', 'low', 'close', 'volume']
            for col in numeric_columns:
                if col in result.columns:
                    result[col] = pd.to_numeric(result[col], errors='coerce')
            
            # Set timestamp as index
            result = result.set_index('_time')
            result = result.sort_index()
            result.index.name = 'timestamp'
            
        return result
        
    except Exception as e:
        print(f"Error fetching aggregated data for {symbol} from {start_date} to {end_date}: {e}")
        return pd.DataFrame()
    
    finally:
        client.close()




In [4]:
def get_all_tickers_ohlcv(start_date=None, end_date=None, interval="1h", max_workers=5):
    """
    Fetch OHLCV data for all available tickers from InfluxDB within a date range
    
    Args:
        start_date (str or datetime): Start date (default: 7 days ago)
        end_date (str or datetime): End date (default: now)
        interval (str): Time interval for aggregation (default: "1h")
        max_workers (int): Maximum number of concurrent threads (default: 5)
    
    Returns:
        dict: Dictionary with ticker symbols as keys and DataFrames as values
              Format: {'BTCUSDT': DataFrame, 'ETHUSDT': DataFrame, ...}
    """
    
    def get_available_tickers(start_date, end_date):
        """Get all unique tickers available in the specified date range"""
        client = InfluxDBClient(url=url, token=token, org=org)
        query_api = client.query_api()
        
        # Handle date formatting
        def format_date(date_input):
            if date_input is None:
                return None
            
            if isinstance(date_input, str):
                if len(date_input) == 10 and date_input.count('-') == 2:
                    date_input += "T00:00:00Z"
                elif not date_input.endswith('Z'):
                    date_input += "Z"
                return date_input
            elif isinstance(date_input, datetime):
                return date_input.strftime('%Y-%m-%dT%H:%M:%SZ')
            else:
                raise ValueError(f"Unsupported date format: {date_input}")
        
        # Set default dates if not provided
        if start_date is None:
            start_str = "-7d"
        else:
            start_str = format_date(start_date)
        
        if end_date is None:
            end_str = "now()"
        else:
            end_str = format_date(end_date)
        
        # Build the time range part of the query
        if start_date is None:
            range_clause = f'|> range(start: {start_str})'
        else:
            if end_date is None:
                range_clause = f'|> range(start: {start_str})'
            else:
                range_clause = f'|> range(start: {start_str}, stop: {end_str})'
        
        # Query to get all unique tickers
        ticker_query = f'''
        from(bucket: "{bucket}")
        {range_clause}
        |> filter(fn: (r) => r._measurement == "klines")
        |> keep(columns: ["symbol"])
        |> distinct(column: "symbol")
        |> group()
        '''
        
        try:
            result = query_api.query_data_frame(ticker_query)
            if not result.empty and 'symbol' in result.columns:
                tickers = result['symbol'].unique().tolist()
                print(f"Found {len(tickers)} tickers in the specified date range")
                return tickers
            else:
                print("No tickers found in the specified date range")
                return []
        except Exception as e:
            print(f"Error fetching tickers: {e}")
            return []
        finally:
            client.close()
    
    def fetch_single_ticker_ohlcv(symbol):
        """Fetch OHLCV for a single ticker"""
        try:
            df = get_ohlcv_aggregated(symbol, start_date, end_date, interval)
            if not df.empty:
                return symbol, df
            else:
                print(f"No data found for {symbol}")
                return symbol, None
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return symbol, None
    
    # Get all available tickers
    print("Fetching available tickers...")
    tickers = get_available_tickers(start_date, end_date)
    
    if not tickers:
        return {}
    
    print(f"Fetching OHLCV data for {len(tickers)} tickers...")
    
    # Dictionary to store results
    ohlcv_data = {}
    
    # Fetch data for all tickers using threading for better performance
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_symbol = {
            executor.submit(fetch_single_ticker_ohlcv, symbol): symbol 
            for symbol in tickers
        }
        
        # Collect results as they complete
        completed = 0
        for future in as_completed(future_to_symbol):
            symbol, data = future.result()
            if data is not None:
                ohlcv_data[symbol] = data
            
            completed += 1
            if completed % 10 == 0 or completed == len(tickers):
                print(f"Completed: {completed}/{len(tickers)} tickers")
    
    print(f"Successfully fetched data for {len(ohlcv_data)} out of {len(tickers)} tickers")
    return ohlcv_data


def get_all_tickers_ohlcv_sequential(start_date=None, end_date=None, interval="1h", delay=0.1):
    """
    Sequential version - use this if you want to avoid overwhelming the database
    
    Args:
        start_date (str or datetime): Start date (default: 7 days ago)
        end_date (str or datetime): End date (default: now)
        interval (str): Time interval for aggregation (default: "1h")
        delay (float): Delay between requests in seconds (default: 0.1)
    
    Returns:
        dict: Dictionary with ticker symbols as keys and DataFrames as values
    """
    
    def get_available_tickers(start_date, end_date):
        """Get all unique tickers available in the specified date range"""
        client = InfluxDBClient(url=url, token=token, org=org)
        query_api = client.query_api()
        
        # Handle date formatting (same as above)
        def format_date(date_input):
            if date_input is None:
                return None
            
            if isinstance(date_input, str):
                if len(date_input) == 10 and date_input.count('-') == 2:
                    date_input += "T00:00:00Z"
                elif not date_input.endswith('Z'):
                    date_input += "Z"
                return date_input
            elif isinstance(date_input, datetime):
                return date_input.strftime('%Y-%m-%dT%H:%M:%SZ')
            else:
                raise ValueError(f"Unsupported date format: {date_input}")
        
        if start_date is None:
            start_str = "-7d"
        else:
            start_str = format_date(start_date)
        
        if end_date is None:
            end_str = "now()"
        else:
            end_str = format_date(end_date)
        
        if start_date is None:
            range_clause = f'|> range(start: {start_str})'
        else:
            if end_date is None:
                range_clause = f'|> range(start: {start_str})'
            else:
                range_clause = f'|> range(start: {start_str}, stop: {end_str})'
        
        ticker_query = f'''
        from(bucket: "{bucket}")
        {range_clause}
        |> filter(fn: (r) => r._measurement == "klines")
        |> keep(columns: ["symbol"])
        |> distinct(column: "symbol")
        |> group()
        '''
        
        try:
            result = query_api.query_data_frame(ticker_query)
            if not result.empty and 'symbol' in result.columns:
                tickers = result['symbol'].unique().tolist()
                print(f"Found {len(tickers)} tickers in the specified date range")
                return tickers
            else:
                print("No tickers found in the specified date range")
                return []
        except Exception as e:
            print(f"Error fetching tickers: {e}")
            return []
        finally:
            client.close()
    
    # Get all available tickers
    print("Fetching available tickers...")
    tickers = get_available_tickers(start_date, end_date)
    
    if not tickers:
        return {}
    
    print(f"Fetching OHLCV data for {len(tickers)} tickers sequentially...")
    
    ohlcv_data = {}
    
    for i, symbol in enumerate(tickers, 1):
        try:
            df = get_ohlcv_aggregated(symbol, start_date, end_date, interval)
            if not df.empty:
                ohlcv_data[symbol] = df
            else:
                print(f"No data found for {symbol}")
            
            if i % 10 == 0 or i == len(tickers):
                print(f"Completed: {i}/{len(tickers)} tickers")
            
            # Add delay to avoid overwhelming the database
            if delay > 0 and i < len(tickers):
                time.sleep(delay)
                
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            continue
    
    print(f"Successfully fetched data for {len(ohlcv_data)} out of {len(tickers)} tickers")
    return ohlcv_data


def save_all_tickers_to_files(ohlcv_data, output_dir="ohlcv_data", file_format="csv"):
    """
    Save all ticker data to separate files
    
    Args:
        ohlcv_data (dict): Dictionary from get_all_tickers_ohlcv()
        output_dir (str): Directory to save files
        file_format (str): File format - "csv", "parquet", or "pickle"
    """
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    for symbol, df in ohlcv_data.items():
        if file_format.lower() == "csv":
            filename = f"{output_dir}/{symbol}_ohlcv.csv"
            df.to_csv(filename)
        elif file_format.lower() == "parquet":
            filename = f"{output_dir}/{symbol}_ohlcv.parquet"
            df.to_parquet(filename)
        elif file_format.lower() == "pickle":
            filename = f"{output_dir}/{symbol}_ohlcv.pkl"
            df.to_pickle(filename)
        else:
            raise ValueError("file_format must be 'csv', 'parquet', or 'pickle'")
    
    print(f"Saved {len(ohlcv_data)} files to {output_dir}/")

def get_all_tickers_ohlcv_with_progress(start_date=None, end_date=None, interval="1h", max_workers=5):
    """
    Enhanced version with better progress tracking
    
    Args:
        start_date (str or datetime): Start date (default: 7 days ago)
        end_date (str or datetime): End date (default: now)
        interval (str): Time interval for aggregation (default: "1h")
        max_workers (int): Maximum number of concurrent threads (default: 5)
    
    Returns:
        dict: Dictionary with ticker symbols as keys and DataFrames as values
    """
    
    def get_available_tickers(start_date, end_date):
        """Get all unique tickers available in the specified date range"""
        client = InfluxDBClient(url=url, token=token, org=org)
        query_api = client.query_api()
        
        # Handle date formatting
        def format_date(date_input):
            if date_input is None:
                return None
            
            if isinstance(date_input, str):
                if len(date_input) == 10 and date_input.count('-') == 2:
                    date_input += "T00:00:00Z"
                elif not date_input.endswith('Z'):
                    date_input += "Z"
                return date_input
            elif isinstance(date_input, datetime):
                return date_input.strftime('%Y-%m-%dT%H:%M:%SZ')
            else:
                raise ValueError(f"Unsupported date format: {date_input}")
        
        # Set default dates if not provided
        if start_date is None:
            start_str = "-7d"
        else:
            start_str = format_date(start_date)
        
        if end_date is None:
            end_str = "now()"
        else:
            end_str = format_date(end_date)
        
        # Build the time range part of the query
        if start_date is None:
            range_clause = f'|> range(start: {start_str})'
        else:
            if end_date is None:
                range_clause = f'|> range(start: {start_str})'
            else:
                range_clause = f'|> range(start: {start_str}, stop: {end_str})'
        
        # Query to get all unique tickers
        ticker_query = f'''
        from(bucket: "{bucket}")
        {range_clause}
        |> filter(fn: (r) => r._measurement == "klines")
        |> keep(columns: ["symbol"])
        |> distinct(column: "symbol")
        |> group()
        '''
        
        try:
            result = query_api.query_data_frame(ticker_query)
            if not result.empty and 'symbol' in result.columns:
                tickers = result['symbol'].unique().tolist()
                print(f"Found {len(tickers)} tickers in the specified date range")
                return tickers
            else:
                print("No tickers found in the specified date range")
                return []
        except Exception as e:
            print(f"Error fetching tickers: {e}")
            return []
        finally:
            client.close()
    
    def fetch_single_ticker_ohlcv(symbol):
        """Fetch OHLCV for a single ticker"""
        try:
            df = get_ohlcv_aggregated(symbol, start_date, end_date, interval)
            if not df.empty:
                return symbol, df
            else:
                return symbol, None
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return symbol, None
    
    # Get all available tickers
    print("Fetching available tickers...")
    tickers = get_available_tickers(start_date, end_date)
    
    if not tickers:
        return {}
    
    print(f"Fetching OHLCV data for {len(tickers)} tickers...")
    start_time = time.time()
    
    ohlcv_data = {}
    failed_tickers = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_symbol = {
            executor.submit(fetch_single_ticker_ohlcv, symbol): symbol 
            for symbol in tickers
        }
        
        completed = 0
        for future in as_completed(future_to_symbol):
            symbol, data = future.result()
            if data is not None:
                ohlcv_data[symbol] = data
                print(f"✓ {symbol}: {len(data)} records")
            else:
                failed_tickers.append(symbol)
                print(f"✗ {symbol}: No data")
            
            completed += 1
            elapsed = time.time() - start_time
            avg_time = elapsed / completed
            eta = avg_time * (len(tickers) - completed)
            
            print(f"Progress: {completed}/{len(tickers)} ({completed/len(tickers)*100:.1f}%) "
                  f"- ETA: {eta:.1f}s")
    
    total_time = time.time() - start_time
    print(f"\nCompleted in {total_time:.1f}s")
    print(f"Success: {len(ohlcv_data)}, Failed: {len(failed_tickers)}")
    if failed_tickers:
        print(f"Failed tickers: {failed_tickers[:10]}..." if len(failed_tickers) > 10 else f"Failed tickers: {failed_tickers}")
    
    return ohlcv_data


In [5]:
start_date,end_date = "2024-01-01","2024-01-31"
btc_data = get_ohlcv_aggregated("BTCUSDT", start_date,end_date,interval="1m")

ohlcv_data = get_all_tickers_ohlcv(start_date=start_date, end_date=end_date, interval="1m", max_workers=5)

Fetching available tickers...


/Users/orentapiero/projects/Crypto/venv/lib/python3.9/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
        from(bucket: "crypto")
        |> range(start: 2024-01-01T00:00:00Z, stop: 2024-01-31T00:00:00Z)
        |> filter(fn: (r) => r._measurement == "klines")
        |> keep(columns: ["symbol"])
        |> distinct(column: "symbol")
        |> group()
         |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flu

Found 216 tickers in the specified date range
Fetching OHLCV data for 216 tickers...
Completed: 10/216 tickers
Completed: 20/216 tickers
Completed: 30/216 tickers
Completed: 40/216 tickers
Completed: 50/216 tickers
Completed: 60/216 tickers
Completed: 70/216 tickers
Completed: 80/216 tickers
Completed: 90/216 tickers
Completed: 100/216 tickers
Completed: 110/216 tickers
Completed: 120/216 tickers
Completed: 130/216 tickers
Completed: 140/216 tickers
Completed: 150/216 tickers
Completed: 160/216 tickers
Completed: 170/216 tickers
Completed: 180/216 tickers
Completed: 190/216 tickers
Completed: 200/216 tickers
Completed: 210/216 tickers
Completed: 216/216 tickers
Successfully fetched data for 216 out of 216 tickers
